In [1]:
%run Futures.ipynb
%run Options.ipynb

In [2]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility as iv
from math import exp, sqrt, log, fabs
import matplotlib.pyplot as plt
from itertools import repeat
import os
import calendar 
import glob

In [3]:
work_dir = os.getcwd()
work_dir
Option_path = os.path.join(work_dir, 'Data', 'Option')
Future_path = os.path.join(work_dir, 'Data', 'Futures')
OptionOpen_path = os.path.join(work_dir, 'Data', 'OptionOpen')
OptionClose_path = os.path.join(work_dir, 'Data', 'OptionClose')
Rate_path = os.path.join(work_dir, 'Data', 'Rate')

In [4]:
Option_path

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy\\Data\\Option'

## Calculate the Date Difference

In [5]:
def dates_delta(start_time, maturity):
    '''
        Compute the distance between two TRADE_DATEs
        Already consider about the weekly options
    '''
    
    s = start_time
    if not isinstance(maturity, str): 
        maturity = str(int(maturity))
    else: 
        maturity = maturity.replace(" ", "")
        
    d0 = date(year = int(s/10000), month = int((s % 10000)/100), day = int(s % 100))
    if maturity[-2] == 'W':
        m = maturity[:5]
        year = int(int(maturity[:6])/100)
        month = int(int(maturity[:6]) % 100)
        weekidx = int(maturity[-1]) - 1
        d1 = get_expireDate(year, month, week = weekidx)
        
    else: 
        year = int(int(maturity[:6])/100)
        month = int(int(maturity[:6]) % 100)
        d1 = get_expireDate(year, month)

    t = np.busday_count(d0, d1)/252
    if t < 0:
        print("The input dates error!")
    return t

In [6]:
def trans_dt(d):
    d_new = date(year = int(d/10000), month = int((d % 10000)/100), day = int(d % 100))
    return d_new

In [7]:
def get_expireDate(year, month, week = 2):
    c = calendar.Calendar(firstweekday=calendar.SATURDAY)
    monthcal = c.monthdatescalendar(year, month)
    monthly_expire_date = monthcal[week][-3]
    return monthly_expire_date

In [8]:
def get_Future_List(): 
    extension = 'csv'
    os.chdir(Future_path)
    Fut_csv_List = glob.glob('*.{}'.format(extension))
    return Fut_csv_List

In [9]:
def get_Option_List(): 
    extension = 'csv'
    os.chdir(Option_path)
    Opt_csv_List = glob.glob('*.{}'.format(extension))
    return Opt_csv_List

In [10]:
def get_zcb_rate(transDate):
    if not isinstance(transDate, date): 
        transDate = np.datetime64(trans_dt(transDate))
    
    rate =  pd.read_csv(os.path.join(Rate_path, 'ZCB.csv'))
    rate['Date'] = pd.to_datetime(rate['Date'])
    zcb = float(rate[rate.Date == transDate]['ZCB'])
    
    return zcb

In [11]:
def MK_disc(S, K, t): 
    'Maturity and Moneyness Discount'
    t = t * 252 # transform into days
    try:
        
        m = float(K/S - 1) # moneyness
        M = max(1, t/30.0) # days to month; at least one-month
        w = exp(-(m**2)/2 - (M - 1)**2)
    except ZeroDivisionError:
        M = max(1, t/30.0)
        w = exp(- (M - 1)**2)
    return w

In [12]:
def call_put_pair(Opt, Fut):
    '''
        Pair the Put Call Options
    '''
    pair_list = []
    pair_dic = {}
    
    length = Opt.get_length()
    
    for i in range(length):
        
        transDate= Opt.get_transDate()
        maturity = Opt.get_maturity(i)
        t = dates_delta(transDate , maturity) 
        K = Opt.get_excercisePrice(i)
        right = Opt.get_right(i)
        interval = Opt.get_interval()
        
        dict_key = "K:" + str(K) + "/" + "T:" + str(t)
        transTime = Opt.get_transTime(i)
        
        S = float(Fut.get_TransPrice(transTime))
        price = Opt.get_transPrice(i)
        
        cur_vol = Opt.get_volumn(i) * MK_disc(S, K, t) # current vol

#=============================================================================
#   Transform the Dict into Pair            
#=============================================================================

        if dict_key not in pair_dic:

            pair_dic[dict_key] = len(pair_dic)
            if right == "C":
                pair_list.append([Opt.df.iloc[i], [], cur_vol, S, 0])
            if right == "P":
                pair_list.append([[], Opt.df.iloc[i], cur_vol, 0, S])
                
        else:
            if right == "C":
                if len(pair_list[pair_dic[dict_key]][0]) < 1:
                    pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                    pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    pair_list[pair_dic[dict_key]][3] = S
                    
                if interval == "Openinterval": 

                    if pair_list[pair_dic[dict_key]][0]['TransTime'] > Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][3] = S
                        
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        
                elif interval == "Closeinterval": 
                    if pair_list[pair_dic[dict_key]][0]['TransTime'] < Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][3] = S
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                
            if right == "P":
                if len(pair_list[pair_dic[dict_key]][1]) < 1:
                    pair_list[pair_dic[dict_key]][1] = Opt.df.iloc[i]
                    pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    pair_list[pair_dic[dict_key]][4] = S
                    
                if interval == "Openinterval": 
                    if pair_list[pair_dic[dict_key]][1]['TransTime'] > Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][1] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][4] = S
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        
                elif interval == "Closeinterval": 
                    if pair_list[pair_dic[dict_key]][1]['TransTime'] < Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][1] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][4] = S
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
    
    return pair_list

In [16]:
def volatility_spread_hour(Opt, Fut):
    volatility_spread = []
    spread_volume = []
    pair_list = call_put_pair(Opt, Fut)
    
    for i in range(len(pair_list)):
        
        if len(pair_list[i][0]) > 1 and len(pair_list[i][1]) > 1: # pair exist
            Sc = float(pair_list[i][3])
            Sp = float(pair_list[i][4])
            Sa = (Sc+Sp)/2.0
         
            transDate = Opt.get_transDate()
            maturity = pair_list[i][0]['Maturity']
            t = dates_delta(transDate , maturity)
            
            r = get_zcb_rate(transDate)*0.01
            K = float(pair_list[i][0]['ExcercisePrice'])
            q = 0
            moneyness = float(K/Sa)
            
            
            call_price = float(pair_list[i][0]['TransPrice'])
            put_price = float(pair_list[i][1]['TransPrice'])
            vol = pair_list[i][2]
            
            PV_K = K*exp(-r*t)
            intrinsic_c = fabs(max(Sc - PV_K, 0.0))
            intrinsic_p = fabs(max(PV_K - Sp, 0.0))
                
            if call_price < intrinsic_c or call_price >= Sc: #Out of the Boundary
                volatility_spread.append(0.0)
                spread_volume.append(0.0)
    
            elif put_price < intrinsic_p or put_price >= K:
                volatility_spread.append(0.0)
                spread_volume.append(0.0)
                
            elif Sc == 0 or Sp == 0: #typo error
                volatility_spread.append(0.0)
                spread_volume.append(0.0) 
                
            elif moneyness > 1.1 or moneyness < 0.9:
                volatility_spread.append(0.0)
                spread_volume.append(0.0) 
                
            elif t > 60/252: 
                volatility_spread.append(0.0)
                spread_volume.append(0.0) 
                
            else:
                try: 
                    call_iv = iv(price = call_price, 
                                 flag = 'c', 
                                 S = Sc, 
                                 K = K, 
                                 t = t, 
                                 r = r,
                                 q = q)

                    put_iv = iv(price = put_price, 
                                 flag = 'p', 
                                 S = Sp, 
                                 K = K, 
                                 t = t, 
                                 r = r,
                                 q = q)

                    cpiv = call_iv - put_iv

                    volatility_spread.append(cpiv)
                    spread_volume.append(vol)
                except ZeroDivisionError:
                    volatility_spread.append(0.0)
                    spread_volume.append(0.0)

                    
        else:
            volatility_spread.append(0.0)
            spread_volume.append(0.0)
            
    try: 
        weights_aggr = [i/sum(spread_volume) for i in spread_volume]
        hour_vs = np.average(volatility_spread, weights = weights_aggr)
        
    except ZeroDivisionError:
        hour_vs = np.nan

    return hour_vs

In [17]:
def saving_file(df, saving_name, results_path = os.path.join(work_dir, "Output_Result")):
    if not os.path.isdir(results_path):
        os.mkdir(results_path)
    results_path = os.path.join(results_path, datetime.today().strftime('%Y%m%d')) 
    print ("writing file to {}".format(results_path))

    if os.path.exists(results_path):
        results_path = results_path + '_' + datetime.today().strftime('_%H%M%S')
    if not os.path.isdir(results_path):
        os.mkdir(results_path)
        
    if isinstance(df, pd.DataFrame):
        df.to_csv(os.path.join(results_path, saving_name), index = False)
    else:
        print("Saving Files Failure!")

In [18]:
if __name__ ==  '__main__':
    Opt_csv_List = get_Option_List()
    Fut_csv_list = get_Future_List()
    length = len(Fut_csv_list)
    datesList = []
    OpenCPIVList = []
    CloseCPIVList = []
    
    for j in range(length): 
        FutCsv = Fut_csv_list[j]
        Fut = Futures(FutCsv)
        datesList.append(Fut.get_transDate())
        
        dataCsv = Opt_csv_List[j]
        OptO = Options(OptionOpen_path, dataCsv)
        OptC = Options(OptionClose_path, dataCsv)
        
        OpenCPIV = volatility_spread_hour(OptO, Fut)
        CloseCPIV = volatility_spread_hour(OptC, Fut)
        
        OpenCPIVList.append(OpenCPIV)
        CloseCPIVList.append(CloseCPIV)
        
        print("Finish {} !".format(datesList[j]))
    
    
    Data_dict = {'Date': datesList, 'OpenIntervalCPIV': OpenCPIVList, 'CloseIntervalCPIV': CloseCPIVList}
    df_CPIV = pd.DataFrame(Data_dict)
    

Finish 20200313 !
Finish 20200316 !
Finish 20200317 !
Finish 20200318 !
Finish 20200319 !
Finish 20200320 !
Finish 20200323 !
Finish 20200324 !


C:\Users\Evan\Anaconda3\lib\site-packages\py_lets_be_rational\lets_be_rational.py:613: RuntimeWarning: divide by zero encountered in double_scalars
  price / (sqrt(F) * sqrt(K)), x, q, N) / sqrt(T)


Finish 20200325 !
Finish 20200326 !
Finish 20200327 !
Finish 20200330 !
Finish 20200331 !
Finish 20200401 !
Finish 20200406 !
Finish 20200407 !
Finish 20200408 !
Finish 20200409 !
Finish 20200410 !
Finish 20200413 !
Finish 20200414 !
Finish 20200415 !
Finish 20200416 !
Finish 20200417 !
Finish 20200420 !
Finish 20200421 !
Finish 20200422 !
Finish 20200423 !
Finish 20200424 !


In [20]:
df_CPIV

,Date,OpenIntervalCPIV,CloseIntervalCPIV
0,20200313,-0.104693,0.051243
1,20200316,0.049534,0.231875
2,20200317,0.010671,0.797703
3,20200318,-0.082681,-0.358737
4,20200319,0.219467,-0.754708
5,20200320,0.290673,0.225436
6,20200323,0.320708,0.305685
7,20200324,0.502459,0.595676
8,20200325,0.042103,0.090247
9,20200326,0.089544,0.056789


In [21]:
saving_file(df_CPIV, saving_name = 'CPIV_0313_0424.csv')

writing file to C:\Users\Evan\Documents\GitHub\TXO_TX_Strategy\Output_Result\20200427
